In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-darkgrid')

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller

import torch
import torch.nn as nn
import torch.nn.functional as F

from metric import mape, mae

In [3]:
# data read
train_df = pd.read_csv('./data/train.csv', encoding='cp949')
test_df = pd.read_csv('./data/test.csv', encoding='cp949')

In [3]:
train_df.head(1)

,num,date_time,elec_amount,temp,wind_speed,humidity,rainfall,sunshine,no_elec,sunlight_have
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0


In [11]:
test_df[test_df['num'] == 1]


,num,date_time,temp,wind_speed,humidity,rainfall,sunshine,no_elec,sunlight_have
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
163,1,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,1,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
165,1,2020-08-31 21,27.1,2.4,74.0,NaN,0.0,NaN,NaN
166,1,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
from copy import deepcopy
copy_test = deepcopy(test_df)

In [5]:
# 결측치 사잇값으로 채우기
interpolate_list = ['temp', 'wind_speed', 'humidity', 'rainfall', 'sunshine']

In [6]:
copy_test[interpolate_list] = copy_test[interpolate_list].interpolate()

In [7]:
# 결측치 train의 대표값으로 채우기(no_elec, sunlight_have)
for i in range(1, 61):
    copy_test.loc[copy_test['num'] == i, 'no_elec'] = copy_test.loc[copy_test['num'] == i, 'no_elec'].fillna(train_df.loc[train_df['num'] == i, 'no_elec'].iloc[0])
    copy_test.loc[copy_test['num'] == i, 'sunlight_have'] = copy_test.loc[copy_test['num'] == i, 'sunlight_have'].fillna(train_df.loc[train_df['num'] == i, 'sunlight_have'].iloc[0])

In [7]:
copy_test

,num,date_time,temp,wind_speed,humidity,rainfall,sunshine,no_elec,sunlight_have
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0


In [7]:
# 대표값으로 결측치 채우기(도연님 ver)
train_df[['num', 'no_elec','sunlight_have']]
ice={}
hot={}
count=0
for i in range(0, len(train_df), len(train_df)//60):
    count +=1
    ice[count]=train_df.loc[i,'no_elec']
    hot[count]=train_df.loc[i,'sunlight_have']

for i in range(len(copy_test)):
    copy_test.loc[i, 'no_elec']=ice[copy_test['num'][i]]
    copy_test.loc[i, 'sunlight_have']=hot[copy_test['num'][i]]

In [ ]:
# 평일, 주말 나누기

def time(x):
    return int(x[-2:])
train_df['time']=train_df['date_time'].apply(lambda x: time(x))
copy_test['time']=copy_test['date_time'].apply(lambda x: time(x))

# 평일=0~4, 주말=5~6
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()
train_df['weekday']=train_df['date_time'].apply(lambda x :weekday(x))
copy_test['weekday']=copy_test['date_time'].apply(lambda x :weekday(x))

# 평일=0, 주말=1
train_df['weekend']=train_df['weekday'].apply(lambda x: 0 if x < 4 else 1)
copy_test['weekend']=copy_test['weekday'].apply(lambda x: 0 if x < 4 else 1)

In [8]:
copy_test.to_csv('pre_test01.csv', index=False)